In [1]:
# Fine tuning gpt2_medium model and use own data like company profile
#
# See also medium.com blog
# "GPT-2 Fine-Tuning Guide: Building a Chatbot for Your Company Profile"
# https://medium.com/@datatec.studio
#

# Mount google driver
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# Change to google driver folder which contains datasets
# This folder will also be used to save model
# %cd /content/drive/MyDrive/GPT2_Lab_DTS

In [3]:
# Install python packages
# !pip install -r requirements.txt

In [4]:
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import time
import numpy as np
import sys

In [5]:
# Define environment variable, path of data, model name and device
# os.environ["HF_HOME"] = "/content/huggingface"  # Replace with your desired directory
# print("Please replace it with your hf access token:")
# os.environ["HF_HOME_TOKEN"] = "Please_replace_it_with_your_hf_access_token"

result_dir = 'resources/'
data_file_name = 'Ester'
data_file_path = f'../data/{data_file_name}.txt'

model_name = "gpt2" # gpt2-medium
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

cuda


In [6]:
# Write a python file to google driver
# Sample of json datasets
# You can also directly upload this code to your google driver
# The code write here in this way is for better understanding of whole project
# %%writefile chat_data.py

from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path: str, tokenizer):
        with open(path, encoding="utf-8") as f:
            self.data = f.readlines()#json.load(open(path, "r"))

        self.X = []
        for pair in self.data:
            pair = eval(pair)          
            self.X.append(f"<start> {pair['prompt']} <response>: {pair['response']} <end>")
        
        total_samples = len(self.X)  # Calculate the total number of samples
        print("total_samples", total_samples)
        # define samples amount
#         self.X = self.X[:500]
        print("Check the preprocessing for self.X[0]:")
        print(self.X[0])

        self.X_encoded = tokenizer(self.X, return_tensors="pt", max_length=30, padding="max_length", truncation=True)
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx]


In [8]:
# Download model, save model and tokernize to harddisk
## prepare tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<start>",
                              "eos_token": "<end>"})

tokenizer.add_tokens(["<response>:"])
with open(data_file_path, encoding="utf-8") as f:
    data = f.readlines()#json.load(open(path, "r"))

for pair in data:
    pair = eval(pair)
#     for token in pair['prompt'].split() + pair['response'].split()
    tokenizer.add_tokens(pair['prompt'].split() + pair['response'].split())
    

## prepare model
### Specify the desired embedding size (must be a multiple of 8)
desired_embedding_size = 50264  # Change this to the desired size
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
### Resize the embedding layer to the desired size
model.resize_token_embeddings(len(tokenizer), desired_embedding_size)
model = model.to(device)

## save tokenizer and model to harddisk
# tokenizer.save_pretrained(result_dir)
# model.save_pretrained(result_dir)

In [9]:
# ## load model and tokenizer from harddisk
# ### Load the GPT-2 tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained(result_dir)

# ### Load the GPT-2 model from the local folder
# model = GPT2LMHeadModel.from_pretrained(result_dir)
# model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(100528, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=100528, bias=False)
)

In [18]:
# Define infer and train function
def infer(inp):
    inp = "<start> " + inp + " <response>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)  # Use .to(device) method to move the tensor to the specified device
    a = inp["attention_mask"].to(device)  # Use .to(device) method here as well

    output = model.generate(X, attention_mask=a, max_length=100, num_return_sequences=1)

    output = tokenizer.decode(output[0])
    output = output[:output.find('<end>')].replace('<pad>', '')
    output = ' '.join(output.split())

    return output

def train(chatData, model, optim):
    
    batches = len(chatData)

    for i, (X, a) in enumerate(chatData):
        X = X.to(device)
        a = a.to(device)
        optim.zero_grad()
        loss = model(input_ids=X, attention_mask=a, labels=X).loss
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'iter {i} out of {batches}')

    print(infer("I created my own branch in our repo"))
    print('========================================================')

# def train(chatData, model, optim):
    
#     batches = len(chatData)

#     for i, (X, a) in tqdm(chatData):
#         X = X.to(device)
#         a = a.to(device)
#         optim.zero_grad()
#         loss = model(input_ids=X, attention_mask=a, labels=X).loss
#         loss.backward()
#         optim.step()
#         if i % 100 == 0:
#             print(f'iter {i} out of {batches}')

#     print(infer("Do we need to bring anything?"))
#     print('========================================================')

In [11]:
# from chat_data import ChatData

#Load ChatData, train model and optimizer
chatData = ChatData(data_file_path, tokenizer)
chatData = DataLoader(chatData, batch_size=1) # batch_size=64

model.train()

optim = Adam(model.parameters(), lr=1e-5)

total_samples 538
Check the preprocessing for self.X[0]:
<start> Thanks Ester! <response>: Ester named the group CSE 151B PA2. <end>


In [12]:
# train 10 times
epochs = 100  # You can adjust the number of epochs as needed
for epoch in range(epochs):
    start = time.time()
    print(f"Epoch {epoch} started")
    train(chatData, model, optim)
    end = time.time()
    print(f"Epoch {epoch} finished in {np.round((end - start) / 60, 2)} minutes")

Epoch 0 started
iter 0 out of 538
iter 100 out of 538
iter 200 out of 538
iter 300 out of 538
iter 400 out of 538
iter 500 out of 538


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<start>  I   created   my   own   branch   in   our   repo   <response>:   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in   in  
Epoch 0 finished in 0.52 minutes
Epoch 1 started
iter 0 out of 538
iter 100 out of 538
iter 200 out of 538
iter 300 out of 538
iter 400 out of 538
iter 500 out of 538


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<start>  I   created   my   own   branch   in   our   repo   <response>:   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the   the  
Epoch 1 finished in 0.51 minutes
Epoch 2 started
iter 0 out of 538
iter 100 out of 538
iter 200 out of 538
iter 300 out of 538
iter 400 out of 538
iter 500 out of 538


...

<start>  I   created   my   own   branch   in   our   repo   <response>:   I   pushed   the   updated   util.py   to   master,   but   I   still   need   a   fix   after   adding   to   the   discussion   section   <end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> <end><pad><pad><pad><pad> <end><pad><pad><pad><pad><pad><pad><pad><pad>  just   fixed  
Epoch 97 finished in 0.51 minutes
Epoch 98 started
iter 0 out of 538
iter 100 out of 538
iter 200 out of 538
iter 300 out of 538
iter 400 out of 538
iter 500 out of 538


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<start>  I   created   my   own   branch   in   our   repo   <response>:   I   created   my   own   branch   in   our   repo   <end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> <end><pad><pad><pad><pad><pad><pad> <end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> <end><pad><pad><pad>
Epoch 98 finished in 0.52 minutes
Epoch 99 started
iter 0 out of 538
iter 100 out of 538
iter 200 out of 538
iter 300 out of 538
iter 400 out of 538
iter 500 out of 538


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<start>  I   created   my   own   branch   in   our   repo   <response>:   I   pushed   the   updated   util.py   version   locally   but   somehow   didn’t   update   on   GitHub   if   you   want   the   DSMLP   GPU   to   be   different  <end><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> 
Epoch 99 finished in 0.52 minutes


In [19]:
inp = ""
while True:
    inp = input("Enter your input (press Enter when done): " + " " * 20)
    print(infer(inp))

Enter your input (press Enter when done):                     Ayo


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<start> Ay o <response>: I’m updating the related works section with citations
Enter your input (press Enter when done):                     bahahah


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<start> b ah ah ah <response>: not submitted yet
Enter your input (press Enter when done):                     are you ok


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<start> Are you Ester? <response>: Are you all free to test your code by cloning my branch code
Enter your input (press Enter when done):                     Who are you?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<start> Wh o are you? <response>: O IM OK NOW <response>: I'm using capstone platform


KeyboardInterrupt: Interrupted by user

In [16]:
# Save the model's state dictionary after training is complete
torch.save(model.state_dict(), f"models/{data_file_name}_ep{epochs}.pt")

In [15]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 633.979MB
